In [10]:
import numpy as np
from scipy import optimize

In [11]:
def obj(theta, nusers, nmovies, X):
    M = X.astype(np.bool)
    u = theta[:nusers].reshape(-1, 1)
    v = theta[nusers:].reshape(-1, 1)
    return np.sum( (M*(u@v.T) - X)**2 )

def fprime(theta, nusers, nmovies, X):
    M = X.astype(np.bool)
    u = theta[:nusers].reshape(-1, 1)
    v = theta[nusers:].reshape(-1, 1)
    vhat = M.T * v
    uhat = M * u
    gradu = np.sum(vhat * (v@u.T - X.T), axis=1).reshape(-1, 1)
    gradv = np.sum(uhat * (u@v.T - X), axis=1).reshape(-1, 1)
    return np.vstack([gradu, gradv])

In [28]:
from dataset import MovieLensDataset
# d = MovieLensDataset('./data/')
d = MovieLensDataset('/Users/nick/test/')
X = d.X.A
M = X.astype(np.bool)

u = np.abs(np.random.randn(d.n_users))
u /= np.linalg.norm(u)
v = np.abs(np.random.randn(d.n_movies))
v /= np.linalg.norm(v)

Loading MovieLens dataset from /Users/nick/test/ with mode sparse..
Column names are userId, movieId, rating, timestamp
Processed 1000 lines. 7 users x 801 movies.
Dataset contains 999 ratings (17.81701444622793% matrix density)


In [29]:
theta0 = np.hstack([u, v])
theta0 = theta0.reshape(-1, 1)
print(theta0.shape)
obj(theta0, d.n_users, d.n_movies, X)

(808, 1)


59190.75180150526

In [114]:
root = optimize.newton(obj, theta0, fprime=fprime, args=(d.n_users, X, ))

IndexError: too many indices for array: array is 0-dimensional, but 2 were indexed

In [33]:
root = optimize.minimize(obj, theta0, args=(d.n_users, X, ))

KeyboardInterrupt: 

# Second order model

In [7]:
def fsecond(theta, m, n, X):
    M = X.astype(np.bool)
    u = theta[:m].reshape(-1, 1)
    v = theta[m:].reshape(-1, 1)
    vhat = M.T * v
    vhat_norm_vect = np.sum(vhat * vhat, axis=1)
    uhat = M * u
    uhat_norm_vect = np.sum(uhat * uhat, axis=1)
    hessian = np.zeros((m+n, m+n), dtype=np.float32) # change to 64
#     print(vhat_norm_vect.shape, uhat_norm_vect.shape)
#     print(hessian.shape, m, n, u.shape, v.shape, (2*(v@u.T)*M.T-X.T).shape, hessian[n:, :m].shape)
    err = 2*(u@v.T)*M-X
    hessian[n:, :n] = err
    hessian[:n, n:] = err.T #2*(v@u.T)*M.T-X.T
    
    # set diag
    row,col = np.diag_indices(hessian.shape[0])
    hessian[row, col] = np.hstack([vhat_norm_vect, uhat_norm_vect])
    
    return hessian

In [149]:
print(fprime(theta0, d.n_users, d.n_movies, X).shape)
fsecond(theta0, d.n_users, d.n_movies, X).shape

(808, 1)


(808, 808)

In [23]:
theta0 = np.random.randn(d.n_movies + d.n_users).reshape(-1, 1)

In [17]:
def rho_f(alpha_):
    return obj(theta+alpha_*direction, d.n_users, d.n_movies, X)

In [22]:
from tqdm import tqdm
theta = theta0
max_iter = 100
m1 = 0.001
delta = 1e-6
for i in tqdm(range(max_iter)):
    grad = fprime(theta, d.n_users, d.n_movies, X)
    gnorm = np.linalg.norm(grad)
#     grad /= gnorm
    print("Grad norm:", gnorm)
    if gnorm < 1e-6:
        print('done!')
        break
    hess = fsecond(theta, d.n_users, d.n_movies, X)
    # compute smallest eigen val of hes (hessian correction)
#     lambdas = np.linalg.eigvals(hess)
    # most computational intensive part O(n^3)
    lambdas, _ = np.linalg.eigh(hess)
#     print('lamda n', np.min(lambdas), lambdas[0])
    eps = max(0., delta-lambdas[0])
    print("Eps:", eps)
    hess += eps * np.eye(theta.shape[0])
    direction = -np.linalg.solve(hess, grad)
#     direction /= np.linalg.norm(direction)
#     print('dir norm', np.linalg.norm(direction))
    
    # backtracking LS + Armijo
    alpha = 1.
    step = .9
    # grad@direction = derivative of rho(0)
#     drho0 = grad.T@direction
#     drho0 /= np.linalg.norm(drho0)
#     print("drho0", drho0)
    alpha = optimize.minimize(rho_f, alpha)
    alpha = alpha.x
#     while obj(theta+alpha*direction, d.n_users, d.n_movies, X) > (obj(theta, d.n_users, d.n_movies, X) + m1*alpha*drho0):
#         alpha *= step
    print('alpha', alpha)
    theta = theta + alpha * direction

  1%|          | 1/100 [00:00<00:13,  7.24it/s]

Grad norm: 103491.23605995566
Eps: 266.9239206777344
alpha [1.03085596e-09]
Grad norm: 103526.99560431429
Eps: 266.56207375390625


  3%|▎         | 3/100 [00:00<00:11,  8.25it/s]

alpha [3.29418257e-10]
Grad norm: 103528.07758721132
Eps: 266.55169777734375
alpha [7.18479453e-10]
Grad norm: 103530.42647152729
Eps: 266.529297875
alpha [-2.04378106e-10]
Grad norm: 103529.95637970655


  6%|▌         | 6/100 [00:00<00:10,  8.77it/s]

...

Eps: 264.92669777734375
alpha [-1.12466646e-10]
Grad norm: 104035.8972024101
Eps: 264.93353371484375
alpha [-1.4294609e-10]
Grad norm: 104035.42788271318
Eps: 264.936768578125


 97%|█████████▋| 97/100 [00:09<00:00,  9.93it/s]

alpha [-4.83098281e-10]
Grad norm: 104033.5095604725
Eps: 264.9500437246094
alpha [3.97713074e-10]
Grad norm: 104040.48091922555
Eps: 264.9019785390625


100%|██████████| 100/100 [00:10<00:00,  9.86it/s]

alpha [-1.63932241e-09]
Grad norm: 104032.2055291869
Eps: 264.959229515625
alpha [6.01095272e-11]
Grad norm: 104032.51097511762
Eps: 264.95709328515625
alpha [-3.35839314e-10]


In [67]:
direction = np.linalg.solve(fsecond(theta0, d.n_users, d.n_movies, X), fprime(theta0, d.n_users, X))
print(direction.shape)

(10334, 1)


In [74]:
grad = fprime(theta, d.n_users, X)
np.linalg.norm(grad)

nan

In [30]:
from scipy import optimize
root = optimize.minimize(obj, theta0, args=(d.n_users, d.n_movies,  X, ))

In [27]:
xx = root.x
grad = fprime(xx, d.n_users, d.n_movies, X)
np.linalg.norm(grad)

8320.651581673017

In [132]:
theta_init = np.abs(np.random.randn(d.n_users+d.n_movies))
sol = optimize.minimize(obj, theta_init, jac=fprime, hess=fsecond, args=(d.n_users, d.n_movies, X, ), options={'gtol': 1e-8, 'disp': True})

/Users/nick/miniconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:522: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


In [133]:
np.linalg.norm(fprime(sol.x, d.n_users, d.n_movies, X))

107013.66903576582

In [16]:
X

array([[ 8,  8,  8, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 8,  0,  0, ...,  0,  0,  0],
       [ 0, 10,  8, ...,  0,  0,  0],
       [ 9,  0,  0, ...,  8,  6,  7]], dtype=uint8)